In [24]:
import pandas as pd
import sqlalchemy
import sqlalchemy.orm
import sqlalchemy.dialects.postgresql as sqlalchemy_pg
import datetime

import os

postgres_url = (
    f"postgresql://{os.getenv('POSTGRES_USER')}:"
    f"{os.getenv('POSTGRES_PASSWORD')}@"
    f"{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/"
    f"{os.getenv('POSTGRES_DB')}?sslmode=require"
)


In [38]:
class Base(sqlalchemy.orm.DeclarativeBase):
    pass

class Events(Base):
    __tablename__ = "events"
    id: sqlalchemy.orm.Mapped[str] = sqlalchemy.orm.mapped_column(sqlalchemy.VARCHAR(32), primary_key=True)
    name: sqlalchemy.orm.Mapped[str] = sqlalchemy.orm.mapped_column(sqlalchemy.String)

    def __repr__(self):
        return f"Event(id={self.id}, name={self.name})"

class Participants(Base):
    __tablename__ = "participants"
    id: sqlalchemy.orm.Mapped[int] = sqlalchemy.orm.mapped_column(sqlalchemy.BigInteger, primary_key=True)
    name: sqlalchemy.orm.Mapped[str] = sqlalchemy.orm.mapped_column(sqlalchemy.String)

    def __repr__(self):
        return f"Participant(id={self.id}, name={self.name})"

class Results(Base):
    __tablename__ = "results"
    event_id: sqlalchemy.orm.Mapped[str] = sqlalchemy.orm.mapped_column(sqlalchemy.VARCHAR(32), sqlalchemy.ForeignKey("events.id"), primary_key=True)
    participant_id: sqlalchemy.orm.Mapped[int] = sqlalchemy.orm.mapped_column(sqlalchemy.BigInteger, sqlalchemy.ForeignKey("participants.id"), primary_key=True)
    result_date: sqlalchemy.orm.Mapped[datetime.date] = sqlalchemy.orm.mapped_column(sqlalchemy.Date, primary_key=True)
    distance: sqlalchemy.orm.Mapped[float] = sqlalchemy.orm.mapped_column(sqlalchemy.NUMERIC)
    steps: sqlalchemy.orm.Mapped[int] = sqlalchemy.orm.mapped_column(sqlalchemy.Integer)

    def __repr__(self):
        return f"Result(event_id={self.event_id}, participant_id={self.participant_id}, result_date={self.result_date}, distance={self.distance}, time={self.time})"


In [39]:
engine = sqlalchemy.create_engine(postgres_url)

Base.metadata.create_all(engine)

In [19]:
historical_results = pd.read_parquet("results.parquet")


In [20]:
historical_results.head()

,event_id,event_name,participant_id,participant_name,date,distance,steps
0,YFHMChFqXHbGXw,Skubiņielas izaicinājums oktobris,276096,Baiba Ruča 🇱🇻,2024-10-30,0.00,0
1,YFHMChFqXHbGXw,Skubiņielas izaicinājums oktobris,276096,Baiba Ruča 🇱🇻,2024-10-29,12.89,18409
2,YFHMChFqXHbGXw,Skubiņielas izaicinājums oktobris,276096,Baiba Ruča 🇱🇻,2024-10-28,14.72,21037
3,YFHMChFqXHbGXw,Skubiņielas izaicinājums oktobris,276096,Baiba Ruča 🇱🇻,2024-10-27,13.55,19364
4,YFHMChFqXHbGXw,Skubiņielas izaicinājums oktobris,276096,Baiba Ruča 🇱🇻,2024-10-26,12.88,18394


In [22]:
events = historical_results[["event_id", "event_name"]].drop_duplicates().rename(columns={"event_id": "id", "event_name": "name"})
events.head()


,id,name
0,YFHMChFqXHbGXw,Skubiņielas izaicinājums oktobris
0,Vh_waa8rbHFAlg,#sept izaicinājums


In [33]:
# Write events to database
insert_stmt = sqlalchemy_pg.insert(Events).values(events.to_dict(orient="records"))
do_update_stmt = insert_stmt.on_conflict_do_update(
    index_elements=["id"],
    set_={c.key: c for c in insert_stmt.excluded}
)

print(do_update_stmt)
engine = sqlalchemy.create_engine(postgres_url)

with engine.connect() as conn:
    conn.execute(do_update_stmt)
    conn.commit()


INSERT INTO events (id, name) VALUES (%(id_m0)s, %(name_m0)s), (%(id_m1)s, %(name_m1)s) ON CONFLICT (id) DO UPDATE SET name = %(param_1)s


In [34]:
participants = historical_results[["participant_id", "participant_name"]].drop_duplicates().rename(columns={"participant_id": "id", "participant_name": "name"})
display(participants.head())

# Write participants to database
insert_stmt = sqlalchemy_pg.insert(Participants).values(participants.to_dict(orient="records"))
do_update_stmt = insert_stmt.on_conflict_do_update(
    index_elements=["id"],
    set_={c.key: c for c in insert_stmt.excluded}
)

print(do_update_stmt)

engine = sqlalchemy.create_engine(postgres_url)

with engine.connect() as conn:
    conn.execute(do_update_stmt)
    conn.commit()

,id,name
0,276096,Baiba Ruča 🇱🇻
0,276112,Linards Kalvāns 🇱🇻
0,276147,Annija Krievina 🇱🇻
0,276105,Līga Ieviņa 🇱🇻
0,276097,Ilze Kalvāne 🇱🇻


INSERT INTO participants (id, name) VALUES (%(id_m0)s, %(name_m0)s), (%(id_m1)s, %(name_m1)s), (%(id_m2)s, %(name_m2)s), (%(id_m3)s, %(name_m3)s), (%(id_m4)s, %(name_m4)s), (%(id_m5)s, %(name_m5)s), (%(id_m6)s, %(name_m6)s), (%(id_m7)s, %(name_m7)s), (%(id_m8)s, %(name_m8)s), (%(id_m9)s, %(name_m9)s), (%(id_m10)s, %(name_m10)s), (%(id_m11)s, %(name_m11)s), (%(id_m12)s, %(name_m12)s), (%(id_m13)s, %(name_m13)s), (%(id_m14)s, %(name_m14)s), (%(id_m15)s, %(name_m15)s), (%(id_m16)s, %(name_m16)s), (%(id_m17)s, %(name_m17)s) ON CONFLICT (id) DO UPDATE SET name = %(param_1)s


In [46]:
# Write results to database

results = historical_results.drop(columns=["event_name", "participant_name"])\
    .rename(columns={"event_id": "event_id", "participant_id": "participant_id", "date": "result_date", "distance": "distance", "steps": "steps"})
display(results.head())

insert_stmt = sqlalchemy_pg.insert(Results).values(results.to_dict(orient="records"))
do_update_stmt = insert_stmt.on_conflict_do_update(
    index_elements=["event_id", "participant_id", "result_date"],
    set_={c.key: c for c in insert_stmt.excluded}
)

print(do_update_stmt)

engine = sqlalchemy.create_engine(postgres_url)

with engine.connect() as conn:
    conn.execute(do_update_stmt)
    conn.commit()

,event_id,participant_id,result_date,distance,steps
0,YFHMChFqXHbGXw,276096,2024-10-30,0.00,0
1,YFHMChFqXHbGXw,276096,2024-10-29,12.89,18409
2,YFHMChFqXHbGXw,276096,2024-10-28,14.72,21037
3,YFHMChFqXHbGXw,276096,2024-10-27,13.55,19364
4,YFHMChFqXHbGXw,276096,2024-10-26,12.88,18394


INSERT INTO results (event_id, participant_id, result_date, distance, steps) VALUES (%(event_id_m0)s, %(participant_id_m0)s, %(result_date_m0)s, %(distance_m0)s, %(steps_m0)s), (%(event_id_m1)s, %(participant_id_m1)s, %(result_date_m1)s, %(distance_m1)s, %(steps_m1)s), (%(event_id_m2)s, %(participant_id_m2)s, %(result_date_m2)s, %(distance_m2)s, %(steps_m2)s), (%(event_id_m3)s, %(participant_id_m3)s, %(result_date_m3)s, %(distance_m3)s, %(steps_m3)s), (%(event_id_m4)s, %(participant_id_m4)s, %(result_date_m4)s, %(distance_m4)s, %(steps_m4)s), (%(event_id_m5)s, %(participant_id_m5)s, %(result_date_m5)s, %(distance_m5)s, %(steps_m5)s), (%(event_id_m6)s, %(participant_id_m6)s, %(result_date_m6)s, %(distance_m6)s, %(steps_m6)s), (%(event_id_m7)s, %(participant_id_m7)s, %(result_date_m7)s, %(distance_m7)s, %(steps_m7)s), (%(event_id_m8)s, %(participant_id_m8)s, %(result_date_m8)s, %(distance_m8)s, %(steps_m8)s), (%(event_id_m9)s, %(participant_id_m9)s, %(result_date_m9)s, %(distance_m9)s, %